# 2.3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought

Auflösung ist ein zentraler Faktor für die OCR Qualität. Entsprechend hilft es, bei älteren Plänen den schriftlichen Teil spaltenweise als Bild abzuspeichern (Spalte = PDF-Seite). Maximale Auflösung => Fehlerquote minimieren. (siehe 2.1.1 bzgl. Nutzungsschablone)

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()

In [ ]:
# A) Textteil – Eine Spalte
tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-TT-crop.png"
ze_tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"

async def run(img_path):
    base64_img, img_size = parser.imageToBase64(img_path)
    print("image_size:", img_size)
    message = jarvis.request([
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_img}",
                "detail": "high"
            }
        },
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
            # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
        }
    ])
    return message

messages = await asyncio.gather(run(ze_tt_path), run(tt_path))
for msg in messages:
    print("#############################################")
    print(msg)

In [3]:
# B) Textteil – Mehrere Spalten
folder_path = "../data/processed/bpläne/1_alles_in_einem_dokument"
img_name = "/L22-01-TT-crop"
tt_prompts = parser.img2prompts(folder_path, img_name)

msg = jarvis.request([
    *tt_prompts,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
        # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
    }
])

print(msg)


3
